<a href="https://colab.research.google.com/github/redjules/Diabetes-Prediction-With-Pyspark/blob/main/Diabetes_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install Dependencies & Run Spark Session

In [ ]:
#install pyspark


In [ ]:
#create a sparksession

## Clone and Export Dataset

In [ ]:
#clone the diabetes dataset from the github repository

In [ ]:
#check if the dataset exists

In [ ]:
#create spark dataframe

In [ ]:
#display the dataframe

In [ ]:
#print the schema


In [ ]:
#count the total no. of diabetic and non-diabetic class

In [ ]:
#get the summary statistics

# Data Cleaning & Preparation

In [ ]:
#check for null values

In [ ]:
#look for the unnecessary values present

# Correlation Analysis & Feature Selection

In [ ]:
#find the correlation among the set of input & output variables

In [ ]:
#feature selection

In [ ]:
#print the schema


In [ ]:
#display dataframe

# Split Dataset & Build the Model

In [ ]:
#create final data

In [ ]:
#print schema of final data

In [ ]:
#split the dataset ; build the model

In [ ]:
#summary of the model

# Evaluate and Save the Model

In [ ]:
# save model

In [ ]:
# load saved model back to the environment

# Prediction on New Data with the saved model

In [ ]:
#create a new spark dataframe

In [ ]:
#print the schema

In [ ]:
#create an additional feature merged column 

In [ ]:
#print the schema

In [ ]:
#use model to make predictions

In [ ]:
#display the predictions